# National Energy Consortium (NEC)

## Problem Statement
- **Objective**: Select one plant form 64 options to serve each demand scenario
- **Goal**: Minimise cost (UDS/MWh) for meeting demand
- **Error Metrics**: RMSE between optimal and selected plant costs
- **Data**: 3 Datasets (demand, plants, generations_costs)

### Error metric

The per-scenario error is defined as:

$$
\text{Error}(d) \;=\; \min_{p \in P} c(p, d) \;-\; c(p'_d, d)
$$

- d: a demand scenario  
- P: set of candidate plants (64 options)  
- p: a plant in P  
- p'_d: the plant selected for scenario d by the model/heuristic  
- c(p, d): cost (UDS/MWh) of plant p under scenario d

Notes:
- This computes the difference between the optimal (minimum) cost across all plants for scenario d and the cost of the selected plant.  
- Use these per-scenario errors to compute aggregate metrics (e.g., RMSE, MAE) across all scenarios.

### Score (RMSE)

The aggregate error score (root-mean-square error) across demand scenarios is:

$$
\text{Score} \;=\; \sqrt{\frac{1}{D}\sum_{d=1}^{D}\text{Error}(d)^2}
$$

Where:
- D: total number of demand scenarios  
- Error(d): per-scenario error (as defined earlier, Error(d) = min_{p in P} c(p,d) - c(p'_d,d))

This score summarizes the typical magnitude of the per-scenario selection error (lower is better).

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import json
import warnings
from dataclasses import dataclass
from typing import Dict, List, Tuple, Optional, Any
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)


#### Helper classes and Data Structure

In [2]:
class Logger:
    def __init__(self, verbose: bool = True):
        self.verbose = verbose
    
    def header(self, text:str, width:int=80):
        if self.verbose:
            print("="*width)
            print(text)
            print("="*width)
    
    def subheader(self,text:str):
        if self.verbose:
            print(f"\n{text}")
            print("-")*len(text)
            
    def info(self, text:str, indent:int=0):
        if self.verbose:
            print(" "*indent + f"✔ {text}")
    
    def data(self, text:str, indent: int=0):
        if self.verbose:
            print(" "*indent + f"➤ {text}")
    
    def metric(self, label:str, value: Any, unit:str="", indent:int=0):
        if self.verbose:
            print(" "*indent +f" {label}: {value} {unit}")
        
    def success(self, text:str):
        if self.verbose:
            print(f"\n{'='*80}")
            print(f"✔ {text}")
            print(f"{'='*80}\n")
        

        
@dataclass
class StepConfig:
    name: str
    description: str
    verbose: bool = True

class Step:
    def __init__(self, config:Optional[StepConfig]=None):
        self.config = config or StepConfig("Step","Generic Step")
        self.logger = Logger(verbose=self.config.verbose) 

    def execute(self, *args, **kwargs):
        raise NotImplementedError("Execute method must be implemented by subclasses")
    
    def get_results(self) -> Dict[str, Any]:
        raise NotImplementedError("get_results method must be implemented by subclasses")

        

## Data Preparation
In the data preparation stage, please adopt a comprehensive approach, addressing the following areas:
1. **Handle any missing data** by identifying and managing incomplete records or missing values to ensure the dataset is ready for further analysis.
2. **Perform relevant feature selection** by determining which features are most important for your goal. Additionally, apply feature scaling to normalise the data and ensure all featues contribute equally to the model, avoiding biases due to deffering ranges.
3. **Focus on identifying the top-performing plants** for each demand by analysing the cost data. Since not all plants perform equally well, you should remove the worst-performing plants from the dataset to make the following tasks more manageable.

In [3]:
class DataPreparation(Step):
    def __init__(self, config:Optional[StepConfig]=None):
        super().__init__(config or StepConfig("Step 1", "Data Preparation"))
        
        ## Results storage
        self.demand_df_clean = None
        self.plants_df_filtered = None
        self.costs_df_filtered = None
        self.demand_features = None
        self.plant_features = None
        self.metadata = {}
        
    @staticmethod
    def validate_features(df:pd.DataFrame, prefix:str)->List[str]:
        features = [col for col in df.columns if col.startswith(prefix) and pd.api.types.is_numeric_dtype(df[col])]
        return features
    
    @staticmethod
    def get_missing_stats(df:pd.DataFrame, columns:List[str])-> Dict[str,Any]:
        total = len(df)
        missing = df[columns].isnull().sum().sum()
        missing_pct = 100 * missing / (len(columns)*total) if total > 0 else 0
        
        ## Calculate mean/std only on numeric columns
        numeric_cols = [col for col in columns if pd.api.types.is_numeric_dtype(df[col])]
        simple_mean = df[numeric_cols].values.mean() if numeric_cols else 0
        simple_std = df[numeric_cols].values.std() if numeric_cols else 0
        
        return {
            "total_records": total,
            "missing_count": missing,
            "missing_percent": missing_pct,
            "feature_count": len(columns),
            "simple_mean": simple_mean,
            "simple_std": simple_std
        }
    
    def fit_transform(self, demand_df:pd.DataFrame, plants_df:pd.DataFrame, demand_features: List[str], plant_features:List[str])-> Tuple[pd.DataFrame, pd.DataFrame]:
        scaler_demand = StandardScaler()
        scaler_plants = StandardScaler()
        
        demand_df_scaled = demand_df.copy()
        plants_df_scaled = plants_df.copy()
        
        demand_df_scaled[demand_features] = scaler_demand.fit_transform(demand_df[demand_features])
        plants_df_scaled[plant_features] = scaler_plants.fit_transform(plants_df[plant_features])
        
        return demand_df_scaled, plants_df_scaled
    
    def analyse_plants(self, costs_df:pd.DataFrame) -> Dict[str, Any]:
        plant_stats = costs_df.groupby("Plant ID")["Cost_USD_per_MWh"].agg(["median","mean", "count"]).sort_values(by="median")
        
        threshold_cost = plant_stats["median"].quantile(75/100)
        worst_plants = plant_stats[plant_stats["median"]> threshold_cost].index.tolist()
        good_plants = plant_stats[plant_stats["median"]<= threshold_cost].index.tolist()
        
        return {
            "stats": plant_stats,
            "threshold": threshold_cost,
            "worst_plants": worst_plants,
            "good_plants": good_plants
        }
    
    def filter_data(self, demand_df:pd.DataFrame, plants_df:pd.DataFrame, costs_df:pd.DataFrame, good_plants:List[str]) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        
        plants_filtered = plants_df[plants_df["Plant ID"].isin(good_plants)].reset_index(drop=True)
        costs_filtered = costs_df[(costs_df["Demand ID"].isin(demand_df["Demand ID"])) & (costs_df["Plant ID"].isin(good_plants))].reset_index(drop=True)
        
        return demand_df, plants_filtered, costs_filtered
    
    def _print_summary(self):
        def _safe_len(obj):
            return len(obj) if obj is not None else 0

        print("Data Preparation Summary:")
        print(f"  - Demand scenarios: {_safe_len(self.demand_df_clean)}")
        print(f"  - Plants: {_safe_len(self.plants_df_filtered)}")
        print(f"  - Cost records: {_safe_len(self.costs_df_filtered)}")
        print(f"  - Demand features: {_safe_len(self.demand_features)}")
        print(f"  - Plant features: {_safe_len(self.plant_features)}")
    
    def get_results(self) -> Dict[str, Any]:
        return {
            "demand_df": self.demand_df_clean,
            "plants_df": self.plants_df_filtered,
            "costs_df": self.costs_df_filtered,
            "demand_features": self.demand_features,
            "plant_features": self.plant_features,
            "metadata": self.metadata
        }
            
    def execute(self, demand_path:str, plants_path:str, costs_path:str):
        
        ## Load data
        demand_df = pd.read_csv(demand_path, keep_default_na=False, na_values=[""])
        plants_df = pd.read_csv(plants_path, keep_default_na=False, na_values=[""])
        costs_df = pd.read_csv(costs_path, keep_default_na=False, na_values=[""])
        
        ## Handle missing values
        demand_feature_cols = self.validate_features(demand_df, "DF")
        self.demand_features = demand_feature_cols
        
        missing_stats = self.get_missing_stats(demand_df, demand_feature_cols)
        
        demand_df_clean = demand_df.copy()
        demand_df_clean[demand_feature_cols] = demand_df_clean[demand_feature_cols].fillna(demand_df_clean[demand_feature_cols].mean())
        
        ## Feature scaling
        plant_feature_cols = self.validate_features(plants_df, "PF")
        self.plant_features = plant_feature_cols
        
        demand_df_scaled, plants_df_scaled = self.fit_transform(demand_df_clean, plants_df, self.demand_features, self.plant_features)
        
        
        ## Remove worst performing plants
        plant_analysis = self.analyse_plants(costs_df)
        
        demand_df_final, plant_df_filtered, costs_df_filtered = self.filter_data(demand_df_scaled, plants_df_scaled, costs_df, plant_analysis["good_plants"])
        
        ## Store results
        self.demand_df_clean = demand_df_final
        self.plants_df_filtered = plant_df_filtered
        self.costs_df_filtered = costs_df_filtered
        
        ## remove NaN cost values
        nan_costs = self.costs_df_filtered["Cost_USD_per_MWh"].isna().sum()
        if nan_costs > 0:
            self.costs_df_filtered = self.costs_df_filtered.dropna(subset=["Cost_USD_per_MWh"])
        
        ## Store metadata
        self.metadata = {
            "demand_featues": self.demand_features,
            "plant_features": self.plant_features,
            "missing_stats": missing_stats,
            "plant_analysis": plant_analysis
        }
        
        ## Print summary
        self._print_summary()


In [4]:
step_one = DataPreparation()
step_one.execute(
    demand_path="Data/raw/demand.csv",
    plants_path="Data/raw/plants.csv",
    costs_path="Data/raw/generation_costs.csv"
)
step_one_results = step_one.get_results()

Data Preparation Summary:
  - Demand scenarios: 500
  - Plants: 48
  - Cost records: 23927
  - Demand features: 12
  - Plant features: 18
